# Image plates of duckweed in growth assay experiment

Ensure the following before running anything in this notebook:
 - The machine is switched on, connected to the computer and all axes have been homed. 
 - The camera tool is accessible and in position 1 on the Jubilee
 
 Once those steps are complete run the cells in this notebook in order

In [15]:
#Importing python libraries downloaded from the internet and those saved locally

import yaml
import matplotlib
matplotlib.use('TkAgg') 
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import json
import threading
from datetime import date
import ipywidgets as widgets
import utils.PlatePositionUtils as pp 


In [ ]:
#This cell will only run if you are working at a computer connected to the Jubilee

from utils.MachineUtils import * 
from utils.CameraUtils import * #Note that this library requirs a package that can only be installed on raspbery pi. 
import utils.ImageCaptureUtils as im

In [10]:
port = '/dev/ttyACM0'
m = MachineCommunication(port)
m.toolChange(1)

## 1. Select config files and directory for image data

### Select config file location

In [16]:
config_file_dirs = ['/Users/Orlando/Documents/Github_clones/duckbot/ConfigFiles', 
                     "/home/pi/Documents/Data/ConfigFiles"]

config_dir_opts = config_file_dirs
dir_choice = widgets.Dropdown(options = config_dir_opts)
display(dir_choice)

Dropdown(options=('/Users/Orlando/Documents/Github_clones/duckbot/ConfigFiles', '/home/pi/Documents/Data/Confi…

In [17]:
expt_config_file_dir = os.path.join(dir_choice.value,'ExptConfigs')

### Select config file
Select the config file that matches the experiment you are collecting data for

In [22]:
os.chdir(expt_config_file_dir)
config_opts = os.listdir(os.getcwd())
expt_choice = widgets.Dropdown(options = config_opts)
display(expt_choice)

Dropdown(options=('Test_082422.json', 'Expt_082222.json', 'Test_expt.json', '.ipynb_checkpoints', 'Test_082422…

In [23]:
print("You selected:")
print(expt_choice.value)
print("If this is not correct re-select before continuing")

You selected:
Test_082422.json
If this is not correct re-select before continuing


In [24]:
expt_setup_filepath = config_file_dir + '/' + expt_choice.value

In [26]:
with open(expt_setup_filepath,'r') as datafile:
    expt_data = json.load(datafile)

#Host directory that you want to host the folder with your image files. End with forward slash
output_data_dir_opts = expt_data["Experimental_data_directories"]
sample_data_dict = expt_data["sample_info"]
expt_name = expt_data["Experiment_name"]

In [28]:
data_dir_choice = widgets.Dropdown(options = output_data_dir_opts)
data_dir_choice.observe( handler = None, names='value')
display(data_dir_choice)

Dropdown(options=('/Users/Orlando/Documents/Github_clones/duckbot/test_data/Test_082422', '/home/pi/Documents/…

In [29]:
Experimental_data_host_directory = data_dir_choice.value

In [31]:
if not os.path.exists(Experimental_data_host_directory):
    print("This path does not exist. Please correct or create required folders and then re-run this cell")
    print(Experimental_data_host_directory)

if os.path.exists(Experimental_data_host_directory):
    data_dir = os.path.join(Experimental_data_host_directory, expt_name)
    if not os.path.exists(data_dir):
        os.mkdir(data_dir) 

## 2. Load plates into machine

In [16]:
#Drop bed down and add plates
plates_to_image = im.pull_plates_to_image(expt_setup_filepath)

m.moveTo(z=100)

print(f'Load the following experimental plates into the appropriate positions on the bed: {plates_to_image}')
print("Add plates to the bed. Hit any button to confirm and proceed")
input()

Load the following experimental plates into the appropriate positions on the bed: ['1', '2', '3']
Add plates to the bed. Hit any button to confirm and proceed
y


'y'

## 3. Focus camera
Run the cells below. It will move the camera over the first well of the first plate and then pop up a live camera window. Manually adjust the focus to the desired point by twisting the housing of the camera lens. You will need to loosen the nut that holds the lens in place to be able to adjust the focus and then tighten it again once you are done. Then hit the escape key to close the window

In [17]:
#Move camera over first well to be imaged
im.move_to_first_well(m, plates_to_image)

In [18]:
#Open a live video window
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

while True:
    ret, frame = cap.read()
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()

cv2.destroyAllWindows()

## 4. Capture and save images

In [32]:
%matplotlib inline

df_with_well_coords = pp.add_well_coords_to_df_from_sample_data_dict(sample_data_dict)

im.image_plates(m, df_with_well_coords, output_data_dir, expt_name)

plates = list(np.unique(df_with_well_coords['Plate']))
plates = [int(x[-1]) for x in plates]

im.create_plate_image_grid(df_with_well_coords, output_data_dir)

In [8]:
m.moveTo(z=100)
print("Please remove plates from machine then hit any key to continue")
input()

Please remove plates from machine then hit any key to continue
h


'h'